In [ ]:
## Load libraries
import numpy as np
import sympy as sp
import sys
import matplotlib.pyplot as plt
import matplotlib.cm as cm
plt.style.use('dark_background')
%matplotlib inline

In [ ]:
## Mount the Google Drive folder, if needed, for accessing data
if('google.colab' in sys.modules):
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    # Change path below starting from /content/drive/MyDrive/Colab Notebooks/
    # depending on how data is organized inside your Colab Notebooks folder in
    # Google Drive
    DIR = '/content/drive/MyDrive/Colab Notebooks/MAHE/MSIS Coursework/OddSem2023MAHE'
    DATA_DIR = DIR+'/Data/'
else:
    DATA_DIR = 'Data/'

---

Calculating the softmax loss symbolically

Consider the following 3 images with 4 features each:

![](https://onedrive.live.com/embed?resid=37720F927B6DDC34%21101114&authkey=%21AEqz7RyEekVGRN8&width=1024)

---

In [ ]:
# Define numpy arrays for each image
x1 = np.array([26, 100, 90, 80]) # Image-1
x2 = np.array([37, 58, 120, 96]) # Image-2
x3 = np.array([12, 30, 46, 54]) # Image-3

# Build symbolic data matrix
X = sp.Matrix(np.stack((x1, x2, x3 ), axis = 1))

# Pretty print symbolic matrix
sp.pprint(X)

In [ ]:
# Create symbolic weight matrix
W = sp.MatrixSymbol('W', 3, 4)

# Pretty print symbolic weight matrix
sp.pprint(sp.Matrix(W)) # Pretty print symbolic weight matrix

In [ ]:
# Create symbolic bias vector
b = sp.MatrixSymbol('b', 3, 1)

# Pretty print symbolic weight matrix
sp.pprint(sp.Matrix(b)) # Pretty print symbolic weight matrix

In [ ]:
# Calculate symbolic raw scores
z0 = W*X[:, 0] + b
z1 = W*X[:, 1] + b
z2 = W*X[:, 2] + b
sp.pprint(z0)

In [ ]:
# Calculate symbolic softmax loss for each image
loss_1 = -sp.log(sp.exp(z0[0,0])/(sp.exp(z0[0,0])+sp.exp(z0[1,0])+sp.exp(z0[2,0])))
loss_2 = -sp.log(sp.exp(z1[1,0])/(sp.exp(z1[0,0])+sp.exp(z1[1,0])+sp.exp(z1[2,0])))
loss_3 = -sp.log(sp.exp(z2[2,0])/(sp.exp(z2[0,0])+sp.exp(z2[1,0])+sp.exp(z2[2,0])))

# Calculate symbolic average softmax loss
average_loss = (1/3)*(loss_1 + loss_2 + loss_3)

# Pretty print symbolic average softmax loss
sp.pprint(average_loss)

In [ ]:
# Evaluate average softmax loss for arbitrary weight and bias values
average_loss.evalf(subs = {W: sp.Matrix(np.random.randn(3,4)), b: sp.Matrix(np.random.randn(3,1))})

---

Function to visualize gradient descent in 1D

---

In [ ]:
## Credit: https://github.com/pablocpz/Gradient-Descent-Visualizations/blob/main/1D%20simulation.ipynb
## Function for visualizing gradient descent in 1D
from matplotlib import animation

def grad_1D(expr, w_values, w_initial, learning_rate, training_epochs, display_animation=False):
    """
    expr : Sympy expression like (w+2)**2+3 (w symbol)
    w_values : np.linspace(a, b, n)
    w_initial: starting value
    learning_rate, training_epochs = z, r
    display_animation : if True, will return two objects to visualizing it (example below)
    """

    w = sp.symbols("w")
    func = sp.lambdify(w, expr, "numpy")
    deriv = sp.diff(expr)
    deriv_func = sp.lambdify(w, deriv, "numpy")

    #algorithm
    local_min = w_initial#np.random.choice(w_values,1)
    initial_local_min = local_min
    print(f"Initial loss value {initial_local_min}")
    model_params = np.zeros((training_epochs, 2)) #shape epochs x 2 cols
    for i in range(0, training_epochs):
        grad = deriv_func(local_min)
        local_min = local_min - (grad*learning_rate)
        model_params[i,0] = local_min
        model_params[i,1] = grad
    print(f"Final loss value {training_epochs} epochs: {local_min}")


    if display_animation:
        #prepare animation
        grad_fig, ax = plt.subplots(figsize=(6, 4), dpi=100)

        ax.plot(w_values, func(w_values), label=f"${sp.latex(expr)}$")
        #ax.plot(w_values, deriv_func(w_values), label=f"dL/dw ${sp.latex(deriv)}$")

        plt.title(f"Empirical Local Minimum: {local_min}")


        plt.axhline(0, color='white',linewidth=0.5)
        plt.axvline(0, color='white',linewidth=0.5)

        plt.grid(color="gray", linestyle="--", linewidth=0.5)
        plt.xlabel("w");
        plt.ylabel("L(w)")
        #plt.legend();
        plt.close()

        def tangent_line(x, x1, y1):
            # m*x+b
            return deriv_func(x1)*(x-x1) + y1

        title = ax.set_title('', fontweight="bold")
        local_min_scat = ax.scatter(initial_local_min, func(initial_local_min), color="orange")
        initial_tangent_range = np.linspace(initial_local_min-0.5, initial_local_min+0.5, 10)
        tangent_plot = ax.plot(initial_tangent_range, tangent_line(x=initial_tangent_range,
                                                                   x1=initial_local_min,
                                                                   y1=func(initial_local_min)), linestyle="--",  color="orange", linewidth=2)[0]
        grad_annotation = ax.annotate(
            'Gradient={0:2f}'.format(deriv_func(initial_local_min)),
            xy=(initial_local_min,func(initial_local_min)), xytext=(initial_local_min,func(initial_local_min)+1),
            arrowprops = {'arrowstyle': "-", 'facecolor' : 'orange'},
            textcoords='data', color='orange' , rotation=20, fontweight="bold"
        )

        def drawframe(epoch):
            title.set_text('Epoch={0:4d}, learning rate = {1:2g}'.format(epoch,  learning_rate))
            x1 = model_params[epoch, 0]
            y1 = func(model_params[epoch, 0])
            local_min_scat.set_offsets((x1, y1))
            tangent_range = np.linspace(x1-0.5, x1+0.5, 10)
            tangent_values = tangent_line(x=tangent_range, x1=x1 ,y1=y1)
            tangent_plot.set_xdata(tangent_range)
            tangent_plot.set_ydata(tangent_values)
            grad_annotation.set_position((x1, y1+1))
            grad_annotation.xy = (x1, y1)
            grad_annotation.set_text('Gradient={0:2f}'.format(model_params[epoch, 1]))
            return local_min_scat,

        # blit=True re-draws only the parts that have changed.

        anim = animation.FuncAnimation(grad_fig, drawframe, frames=training_epochs, repeat=False, interval=500, blit=True)


        writer = animation.PillowWriter(fps=30,
                                        metadata=dict(artist='Me'),
                                        bitrate=1800)
        # ani.save('gradient1D.gif', writer=writer)
        # from IPython.display import HTML
        # HTML(anim.to_html5_video())

        return anim, writer

---

Gradient descent visualization in 1D applied to $L(w) = (w+2)^2+3$ starting from $w=2.$

---

In [ ]:
## Generate animation for gradient descent in 1D
w = sp.symbols('w')
anim, writer = grad_1D(expr=(w+2)**2+3,
                       w_values=np.linspace(-6,6, 20),
                       w_initial = 2.0, learning_rate=0.1,
                       training_epochs=100,
                       display_animation=True)
#anim.save(DATA_DIR+'gradient1D.gif', writer=writer) #if you want to save it as GIF
from IPython.display import HTML
HTML(anim.to_html5_video()) #render the video

---

Gradient descent in 1D applied to $L(w) = (w+2)^2+3$ starting from $w=2.$

----

In [ ]:
## Gradient descent in 1D
L = lambda w: (w+2)**2 + 3
gradL = lambda w: 2*(w+2)
# Try 1e-05 (slow learning rate), 1e-01 (optimal),
# 1e0 (oscillates and does not converge),
# and 0.95 (oscillates towards the end and converges)
alpha = 0.95 # learning rate
tol = 1e-05 # stopping tolerance
iter = 0
maxiter = 1000

w = 2 # starting point

# Learning process
while np.abs(gradL(w)) > tol and iter < maxiter:
  w = w + alpha * -gradL(w)
  iter = iter+1
  print('Iteration = %d, w = %f, gradL(w) = %f'%(iter, w, gradL(w)))

---

Gradient Descent in 2D applied to $L(\mathbf{w}) = (w_1-2)^2+(w_2+3)^2$ starting from $w_1=1, w_2=1.$

---

In [ ]:
# Gradient descent in 2D
L = lambda w: (w[0]-2)**2 + (w[1]+3)**2
gradL = lambda w: np.array([2*(w[0]-2), 2*(w[1]+3)])
alpha = 1e-02 # learning rate
tol = 1e-05 # stopping tolerance
iter = 0
maxiter = 1000

w =  np.array([1, 1]) # initial guess

while np.linalg.norm(gradL(w)) > tol and iter < maxiter:
  w = w + alpha *(-gradL(w))
  iter = iter+1
  print('Iteration = %d, w1 = %f, w2 = %f, ||gradL(w)|| = %f'%(iter, w[0], w[1], np.linalg.norm(gradL(w))))

---

Calculating softmax loss and gradient for a toy dataset

----

In [ ]:
# Generate artificial data with 5 samples, 4 features per sample
# and 3 output classes
num_samples = 5 # number of samples
num_features = 4 # number of features (a.k.a. dimensionality)
num_labels = 3 # number of output labels
# Data matrix (each column = single sample)
X = np.random.choice(np.arange(0, 5), size = (num_features, num_samples), replace = True)
# Class labels
y = np.random.choice([0, 1, 2], size = num_samples, replace = True)
# Randomly assign entries of weights matrix
W = np.random.choice(np.arange(-4, 4), size = (num_labels, num_features), replace = True)
print('X = ')
print(X)
print('y = ')
print(y)
print('W = ')
print(W)

In [ ]:
# Add the bias feature to the data matrix (run this cell only once!)
print('X = ')
print(X)
print('X with bias feature = ')
X = np.vstack([X, np.ones((1, num_samples))])
print(X)

In [ ]:
# Adjust the weight matrix with (possibly random) values added
# for bias as the last column (run this cell only once!)
W = np.hstack([W, np.ones((num_labels, 1))])
print(W)

In [ ]:
# Calculate the raw zcores matrix
Z =  np.dot(W, X)
print('Z = ')
print(Z)

In [ ]:
# Convert scores to non-normalized probabilites matrix. Note that for each sample,
# that is in each column, the values don't add up to 1. Also note that the
# output values are typically large or small
P = np.exp(Z)
print('P = ')
P

In [ ]:
# Normalize probabilities matrix such that the sum across each column is equal to 1.
# Now we have actually probability values for each sample.
P = P / np.sum(P, axis = 0)
print('P = ')
print(P)
# Sum in each column of matrix P
print(np.sum(P, axis = 0))
# Print the correct label for each sample
print(y)

In [ ]:
# Calculate loss for all samples.
loss =  -np.log(P[y, np.arange(num_samples)])
print('Loss = ')
print(loss)
# Calculate average training loss
loss_data = np.mean(loss)
print('Total loss = %f'%(loss_data))

In [ ]:
# Adjust the probability matrix such that 1 is subtracted
# from each samples correct category probability
P[y, range(num_samples)] = P[y, range(num_samples)] - 1

In [ ]:
# Calculate the gradient of total loss w.r.t. the weights W
dW = (1/num_samples)*np.dot(P, X.T)
print(dW)

---

Applying $L_2$-regularization (loss and gradient)

---

In [ ]:
# Generate artificial data with 5 samples, 4 features per sample
# and 3 output classes
num_samples = 5 # number of samples
num_features = 4 # number of features (a.k.a. dimensionality)
num_labels = 3 # number of output labels
# Data matrix (each column = single sample)
X = np.random.choice(np.arange(0, 5), size = (num_features, num_samples), replace = True)
# Class labels
y = np.random.choice([0, 1, 2], size = num_samples, replace = True)
# Randomly assign entries of weights matrix
W = np.random.choice(np.arange(-4, 4), size = (num_labels, num_features), replace = True)
# Add the bias feature to the data matrix
X = np.vstack([X, np.ones((1, num_samples))])
# Adjust the weight matrix with (possibly random) values added
# for bias as the last column
W = np.hstack([W, np.ones((num_labels, 1))])
# Calculate the raw zcores matrix
Z =  np.dot(W, X)
# Convert scores to non-normalized probabilites matrix. Note that for each sample,
# that is in each column, the values don't add up to 1. Also note that the
# output values are typically large or small
P = np.exp(Z)
# Normalize probabilities matrix such that the sum across each column is equal to 1.
# Now we have actually probability values for each sample.
P = P / np.sum(P, axis = 0)

In [ ]:
# Calculate loss for all samples.
loss =  -np.log(P[y, np.arange(num_samples)])
print('Loss = ')
print(loss)
# Calculate total average data loss
loss_data = np.mean(loss)
# Regularization loss
alpha = 0.1 # strength of regularization = 10%
loss_reg = np.sum(W[:, :-1] * W[:, :-1])
print('Total loss = %f'%(loss_data + alpha*loss_reg))

In [ ]:
# Adjust the probability matrix such that 1 is subtracted
# from each samples correct category probability
P[y, range(num_samples)] = P[y, range(num_samples)] - 1

In [ ]:
# Calculate the gradient of total loss w.r.t. the weights W
dW = (1/num_samples)*np.dot(P, X.T) +2 * alpha* np.hstack([W[:, :-1], np.zeros((num_labels, 1))])
print(dW)

---

Batch Processing

---

In [ ]:
# Demonstration of splitting samples into batches for batch processing
# using a simple example

num_samples = 11 # total number of samples
num_iters = 10   # number of iterations
batch_size = 3   # number of samples for calculating loss and gradient in each iteration

print('Number of samples = %d'%(num_samples))
print('Number of iterations = %d'%(num_iters))
print('Batch size = %d'%(batch_size))

# Function to generate sample indices for batch processing according to batch size
def generate_batch_indices():
  # Reorder sample indices
  reordered_sample_indices = np.random.choice(num_samples, num_samples, replace = False)
  # Generate batch indices for batch processing
  batch_indices = np.split(reordered_sample_indices, np.arange(batch_size, len(reordered_sample_indices), batch_size))
  return(batch_indices)

# Number of batches per epoch
num_iterations_per_epoch = int(np.ceil(num_samples/batch_size))
print('Number of iterations per epoch = %d\n'%(num_iterations_per_epoch))
b = 0
epoch = 0
for it in range(num_iters):
  if it % num_iterations_per_epoch == 0:# check if we are at the start of an epoch
    print('--------------------------------')
    print('Epoch %d:'%(epoch+1))
    batch_indices = generate_batch_indices()
    b = 0
    epoch = epoch + 1
    print('--------------------------------')
  print('In iteration %d, using samples' % (it+1))
  print(batch_indices[b])
  b += 1